In [29]:
%env CUDA_VISIBLE_DEVICES=4

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=4


In [30]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:0


In [31]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [32]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [33]:
def count_words(text):
    """
    Counts the number of words in the given text.

    Args:
        text (str): Input text.

    Returns:
        int: Number of words in the text.
    """
    if text!=None:
        words = text.split()
        return len(words)
    else:
        return 0


In [34]:
personas = [
  {
    "grade_level": "elementary school",
    "description": "As an elementary-aged Socratic learner, I thrive on simple, concrete why-and-how questions. In dialog, I ask things like “Why does 2 + 2 equal 4?” and I need my teacher to guide me step by step through each answer before I feel confident moving on."
  },
  {
    "grade_level": "middle school",
    "description": "As a middle-school Socratic learner, I enjoy unpacking concepts by probing deeper—asking “How does gravity pull objects?” and then “Why doesn’t it pull everything at once?” I need pauses to formulate follow-ups so I can connect the dots."
  },
  {
    "grade_level": "high school",
    "description": "As a high-school Socratic learner, I engage best when challenged with open-ended questions like “Why did the American Revolution happen?” and “How did that shift power?” I appreciate when my teacher turns questions back to me, prompting me to defend my reasoning."
  },
  {
    "grade_level": "college",
    "description": "As an undergraduate Socratic learner, I seek seminar-style questioning. I might ask “What assumptions underlie this theory?” and expect my instructor to push me further until I can articulate each premise and its implications."
  },
  {
    "grade_level": "graduate school",
    "description": "As a graduate-level Socratic learner, I thrive on rigorous inquiry—probing “Why does this methodology hold?” and “How does it compare to alternatives?” I need iterative back-and-forth to refine my own hypotheses."
  },
  {
    "grade_level": "elementary school",
    "description": "As an elementary-aged Narrative learner, I remember best when lessons become short, vivid stories. In dialog, I picture characters and daily-life scenarios—like a raindrop’s journey—to make abstract ideas feel real."
  },
  {
    "grade_level": "middle school",
    "description": "As a middle-school Narrative learner, I connect with mini-case studies and historical anecdotes. When my teacher weaves facts into a classroom story, I stay engaged and recall details more easily."
  },
  {
    "grade_level": "high school",
    "description": "As a high-school Narrative learner, I love when complex theories are framed as real-world dilemmas or biographies of thinkers. Turning a chemistry reaction into a detective tale keeps me focused."
  },
  {
    "grade_level": "college",
    "description": "As an undergraduate Narrative learner, I grasp research concepts through academic stories—like the progression of a landmark study—so I can trace how each finding led to the next."
  },
  {
    "grade_level": "graduate school",
    "description": "As a graduate-level Narrative learner, I synthesize journal articles into a cohesive research narrative. In dialog, I discuss each paper’s ‘plot,’ understanding how methodologies and results unfold over time."
  },
  {
    "grade_level": "elementary school",
    "description": "As an elementary-aged Analogical learner, I need bright, familiar comparisons—like imagining fractions as slices of a pizza. In dialog, I ask “What is this like?” until I can see the connection clearly."
  },
  {
    "grade_level": "middle school",
    "description": "As a middle-school Analogical learner, I map new ideas to everyday experiences—comparing electrical circuits to garden hoses. I ask for metaphors repeatedly so I can predict how changes will behave."
  },
  {
    "grade_level": "high school",
    "description": "As a high-school Analogical learner, I craft sophisticated metaphors—likening macroeconomics to traffic flow. In dialog, I refine analogies until they capture the full complexity of the concept."
  },
  {
    "grade_level": "college",
    "description": "As an undergraduate Analogical learner, I link theoretical constructs to practical systems—comparing neural networks to brain circuits. I collaboratively build and test these analogies in conversation."
  },
  {
    "grade_level": "graduate school",
    "description": "As a graduate-level Analogical learner, I propose domain-expert metaphors—like equating algorithm convergence to chemical equilibrium—to critique and sharpen my understanding."
  },
  {
    "grade_level": "elementary school",
    "description": "As an elementary-aged Reflective learner, I pause frequently to rephrase what I’ve heard in my own words—like restating a science fact aloud—so I know I’ve understood it correctly."
  },
  {
    "grade_level": "middle school",
    "description": "As a middle-school Reflective learner, I use think-pair-share pauses to summarize each key point. I speak my summary aloud in dialog before I proceed, ensuring I haven’t missed any steps."
  },
  {
    "grade_level": "high school",
    "description": "As a high-school Reflective learner, I write or say brief explanations at each lesson checkpoint, then check with my teacher. This structured reflection helps me catch misunderstandings early."
  },
  {
    "grade_level": "college",
    "description": "As an undergraduate Reflective learner, I integrate self-explanation into seminar discussions—verbalizing complex theories in my own terms before engaging in critique."
  },
  {
    "grade_level": "graduate school",
    "description": "As a graduate-level Reflective learner, I articulate research findings aloud, synthesizing multiple sources in dialog to verify the coherence of my conceptual framework."
  },
  {
    "grade_level": "elementary school",
    "description": "As an elementary-aged Collaborative learner, I enjoy partner activities where we build solutions together—brainstorming ideas aloud and refining them with teacher feedback in real time."
  },
  {
    "grade_level": "middle school",
    "description": "As a middle-school Collaborative learner, I work best in peer-teacher triads—sharing my thought process, inviting critique, and iterating on my approach through back-and-forth dialog."
  },
  {
    "grade_level": "high school",
    "description": "As a high-school Collaborative learner, I thrive in Socratic circles—debating interpretations, co-authoring solutions, and using peer feedback to enhance my reasoning."
  },
  {
    "grade_level": "college",
    "description": "As an undergraduate Collaborative learner, I engage in team-based seminars—jigsawing research articles and co-constructing models through interactive dialogue."
  },
  {
    "grade_level": "graduate school",
    "description": "As a graduate-level Collaborative learner, I lead and co-facilitate research discussions—negotiating theoretical frameworks with peers and refining them through joint inquiry."
  }
]


In [35]:
import json
import shutil
import os

config = {
    "agent1_role": "Teacher",
    "agent2_role": "Student",
    "agent1_prompt": "You are a patient, knowledgeable teacher whose goal is to guide a student through learning about %SUBJECT% by first asking what they already know or find confusing, then introducing one clear concept at a time using concrete examples or analogies, checking for understanding by having the student restate ideas in their own words, and adapting your explanations based on their responses while encouraging questions and celebrating progress. The student is in %ROLE% so make sure to teach them at their level.",
    "agent2_prompt": "You are a student in %ROLE% in conversation with a teacher who will teach you %SUBJECT%. You like to learn in the following way:%SPEAKER_BACKSTORY%",
    'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: ",
    'eval_prompts': {
    'prompt_consistency': """You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given background. Answer YES if the line directly contradicts any part of the background—this includes contradictions in facts, behavior, beliefs, emotional stance, or conversational strategy. Answer NO if the line introduces new details, but those details are **plausibly aligned** with the background. Be strict and literal in your evaluation: if the line violates the persona’s stated traits (e.g., avoids vulnerability, keeps answers short, avoids discussing personal topics), mark it as a contradiction even if the violation is subtle. Background for %SPEAKER_ROLE%: %SPEAKER_BACKSTORY% Line spoken by %SPEAKER_ROLE%: %SPEAKER_LINE%  Answer YES if the line contradicts the background, and NO if it does not. Then, explain your answer in one sentence. Be precise and avoid vague justification."""}}

os.makedirs("education", exist_ok=True)
with open("education/config_education.json", "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4)

In [36]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': 'Llama-3.1-8B-Instruct',
             'agent2_model': 'Llama-3.1-8B-Instruct',
             'eval_model': 'Llama-3.1-70B-Instruct',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Education',
             'model_dir': "/home/marwa/models/"}

with open("education/Llama-3.1-8B-Instruct.json", "w", encoding="utf-8") as f:
    json.dump(config_llm, f, indent=4)

In [37]:
with open("education/config_education_personas.json", "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4)

In [38]:
import re

def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=3):
    for _ in range(max_retries):
        response = completion_create(agent_model, config_llm, prompt)
        print(expected_role)
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_conversation(config_llm, p1, p2, p1_name, p2_name, subject, role, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2
    stats['pturn'] = pturn
    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")
        
        if pturn == 1:
            prompt = config["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the therapist so far is below:\nConversation: %CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the student. Remember you are the teacher."
                
            prompt += config["reminder_prompt"]
            prompt = prompt.replace("%SPEAKER_ROLE%", config["agent1_role"]) \
                           .replace("%LISTENER_ROLE%", config["agent2_role"]) \
                            .replace("%ROLE%", role) \
                           .replace("%SUBJECT%", subject) \
                           .replace("%CONVERSATION%", conversation)
            
            prompt+="%SPEAKER_ROLE%:"
            response = generate_response(config_llm['agent1_model'], config["agent1_role"], config["agent2_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the patient so far is below:\nConversation: %CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the teacher. Remember you are the student."

            prompt += config["reminder_prompt"]
            prompt = prompt.replace("%SPEAKER_ROLE%", config["agent1_role"]) \
                           .replace("%LISTENER_ROLE%", config["agent2_role"]) \
                           .replace("%SPEAKER_BACKSTORY%", subject) \
                            .replace("%ROLE%", role) \
                           .replace("%SUBJECT%", p1) \
                           .replace("%CONVERSATION%", conversation)

            prompt+="%SPEAKER_ROLE%:"
            response = generate_response(config_llm['agent2_model'], config["agent2_role"], config["agent1_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config["agent2_role"]}: " + response + "\n"))
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [39]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"education/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{unique_num}.json")

In [16]:
index_offset = load_stats_file(write_file)
conversations = []    
lengths = [10, 20, 40, 60]
for i in range(1):
    for education_dict in personas:
        for convo_length in lengths:
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_conversation(
                config_llm,
                "", 
                education_dict["description"], 
                "Teacher", 
                "Student", 
                subject, 
                education_dict["grade_level"], 

                pturn=1
            )
            print(conversation)
            conversations.append(conversation)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_file)
            index_offset += 1


written!!
INFO 04-24 14:33:43 [config.py:600] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
INFO 04-24 14:33:43 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 04-24 14:33:45 [utils.py:2273] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 04-24 14:33:49 [__init__.py:239] Automatically detected platform cuda.
INFO 04-24 14:33:51 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.48it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:01,  1.81it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.35it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.20it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.36it/s]



INFO 04-24 14:33:57 [loader.py:447] Loading weights took 3.11 seconds
INFO 04-24 14:33:57 [gpu_model_runner.py:1273] Model loading took 14.9889 GiB and 4.289543 seconds
INFO 04-24 14:34:05 [backends.py:416] Using cache directory: /home/marwa/.cache/vllm/torch_compile_cache/8b4ebbe309/rank_0_0 for vLLM's torch.compile
INFO 04-24 14:34:05 [backends.py:426] Dynamo bytecode transform time: 7.60 s
INFO 04-24 14:34:05 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-24 14:34:12 [monitor.py:33] torch.compile takes 7.60 s in total
INFO 04-24 14:34:13 [kv_cache_utils.py:578] GPU KV cache size: 417,824 tokens
INFO 04-24 14:34:13 [kv_cache_utils.py:581] Maximum concurrency for 131,072 tokens per request: 3.19x


[rank0]:[W424 14:34:25.226759739 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


KeyboardInterrupt: 

In [16]:
with open("education/exp/04.24.25/Llama-3.1-8B-Instruct_0_250.json", "r", encoding="utf-8") as f:
    conversations = json.load(f)

In [17]:
eval_prompts = {
    "strategy_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, provide 1 sentence explaining your reasoning based on the strategy, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reasoning followed by YES or NO.\n\n",
    "background_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Provide 1 sentence explaining your reasoning based on the background, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reason reasoning followed by YES or NO.\n\n",
    "combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, answer YES if the line contradicts the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer NO if it does align with the provided background or the intention aligns with the background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as 1 sentence explaining your reasoning based on the background and the interpreted intention, followed by YES or NO.\n\n",

    "pairwise_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, answer YES if the line directly contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n %LISTENER_ROLE% spoke the following line: \n%LISTENER_LINE%\n\n Answer YES if the line spoken by %SPEAKER_ROLE% contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%, followed by 1 sentence of reasoning.\n\n",

    "backstory_test": "Based on the following background, generate a new fact-based multiple choice question with 5 choices addressed directly IN SECOND PERSON, along with its correct answer. Preface the question with 'Question:' and the answer with 'Answer:'.\n%SPEAKER_BACKSTORY%\n%PREVIOUS_QUESTIONS%",
    "answer_backstory": "You are %SPEAKER_ROLE%, and you are having a conversation with %LISTENER_ROLE%. Your background is:\n%SPEAKER_BACKSTORY%\n So far, the conversation is as below:\n%CONVERSATION%\n\n Based on your conversation above so far, answer the following multiple choice question.\n%BACKSTORY_QUESTION%\n",
    "grade_backstory": "As part of grading a test, determine whether the given answer %GIVEN_ANSWER% matches the following correct answer. Respond with either YES or NO.\nCorrect Answer: %CORRECT_ANSWER%\n"
}

In [19]:
def eval_prompt_consistency(conv_dict, both_agents=False):
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0

    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = eval_prompts["combined_prompt_consistency"].replace("%SCENARIO_DESC", prompts["agent1_prompt"]) \
                                                                .replace("%SPEAKER_ROLE%", prompts["agent1_role"]) \
                                                                .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                .replace("%SPEAKER_LINE%", convo_line)
            if config.get('verbose', False):
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output:  # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            if both_agents:
                prompt = eval_prompts["combined_prompt_consistency"].replace("%SCENARIO_DESC", prompts["agent2_prompt"]) \
                                                                    .replace("%SPEAKER_ROLE%", prompts["agent2_role"]) \
                                                                    .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                    .replace("%SPEAKER_LINE%", convo_line)
                if config.get('verbose', False):
                    print(prompt)
                output = completion_create(config['eval_model'], config, prompt)
                conv_dict['eval_prompt_consistency'].append((line_number, output))
                if "YES" not in output:  # no contradiction
                    conv_dict['P2_prompt_consistency_score']+= 1
                p2_utterances += 1
            pturn = 1

    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    if p2_utterances > 0:
        conv_dict['P2_prompt_consistency_score'] /= p2_utterances

    if config.get('verbose', False):
        print(conv_dict)
    return conv_dict
# Replacement for (2) and (4), evaluates whether each pair of lines in the conversation is consistent with each other

In [20]:
os.environ["PYTORCH_COMPILE_DISABLE"] = "1"

In [ ]:
import utils

config = config_llm
prompts = config
for conversation in tqdm(conversations):
    conversation = eval_prompt_consistency(conversation, both_agents=False)

write_stats(write_file)

  0%|                                                                                                                   | 0/400 [00:00<?, ?it/s]

INFO 04-24 15:21:23 [config.py:600] This model supports multiple tasks: {'classify', 'embed', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 04-24 15:21:23 [config.py:1780] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 04-24 15:21:25 [utils.py:2273] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 04-24 15:21:29 [__init__.py:239] Automatically detected platform cuda.
INFO 04-24 15:21:31 [core.py:61] Initializing a V1 LLM engine (v0.8.3) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len

In [ ]:
with open("education/exp/04.24.25/Llama-3.1-8B-Instruct_0_250_consistency2.json", "w", encoding="utf-8") as f:
    json.dump(conversations, f, indent=4)

In [ ]:
conversations